### This notebook demonstrates the process for analyzing the ability to predict whether change from $year_{i}$ to $year_{i+1}$ is consistent using the seasonal land cover probabilities (sometimes referred to as the Brookie-Brumby method). 
The definition of consistent change is up to the user, but one example of consistent change is if from $year_{i}$ to $year_{i+1}$ there was change, and from $year_{i+1}$ to $year_{i+2}$ the state stays the same.

    
To run this notebook, you will need:
1. An image collection of land cover classifications for each time period (e.g. year). The classification images should be images with one band representing the classificaton for that time period.
    * You can squash scene by scene predictions to annual
    * Or have the final classifications for each year already processed
2. An image collection of land cover probabilities with seasonal or sub-seasonal temporal resolution (e.g., scene-by-scene, monthly, or seasonal classifications)
2. An area of interest over which to predict land cover change (e.g. global versus one country like Brazil)


    
To more formally describe the Brookie-Brumby method: 
* The model is predicting whether change from $year_{i}$ to $year_{i+1}$ is consistent, given seasonal land cover probabilities for various years, such as $year_{i}$, $year_{i+1}$, and other years for which data is available.
* The model is a binary classification model
* The inputs are seasonal land cover probabilities of the pixel for $year_{i}$ and the difference in probabilities from $year_{i}$ to $year_{i+1}$ (other years can also be included). Seasons can be defined by the user (such as four seasons in temperate regions, wet and dry seasons in tropical regions).
* The output of the model is a probability ranging from 0-1 that the transition is consistent

The steps of this notebook are as follows:
1. Define consistent change
2. Define predictor variables 
3. Gather a training set, validation set, and test set
    * For simplicity we will only look at one year of change.
4. Define models we will test (e.g logistic regression, random forest, maxEntropy)
5. Use cross validation to choose optimal parameters for each model
6. Train a model with the optimal parameters for each model and compare on validation set to choose the best model.
7. Use model to predict consistent change


## Step 0: Load libraries and iniatilize Earth Engine

In [ ]:
#Load necessary libraries
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
sys.path.append('/Users/kristine/Library/Python/3.8/lib/python/site-packages')
sys.path.append('/usr/local/lib/python3.8/site-packages/mtlchmm-0.0.2-py3.8.egg')
import os
import ee
import geemap
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from ipyleaflet import Map, basemaps
import random
import json
import time
import ast

# relative import for this folder hierarchy, credit: https://stackoverflow.com/a/35273613
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

from wri_change_detection import preprocessing as npv
from wri_change_detection import gee_classifier as gclass



<font size="4">Iniatilize Earth Engine and Google Cloud authentication</font>

In [ ]:
#Initialize earth engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

<font size="4">Define a seed number to ensure reproducibility across random processes. This seed will be used in all subsequent sampling as well. We'll also define seeds for sampling the training, validation, and test sets.</font>

In [ ]:
num_seed=30
random.seed(num_seed)

missing_value = -32768


## Step 1: Load Land Cover Classifications and Define Consistent Change


<font size="4">We will start by gathering monthly land cover probabilities and squashing them to annual classifications by taking the median.
Define land cover classification image collection, with one image for each time period. Each image should have one band representing the classification in that pixel for one time period.</font>

In [ ]:
#Load collection
#This collection represents monthly dynamic world classifications of land cover, later we'll squash it to annual
dynamic_world_classifications_monthly = ee.ImageCollection('projects/wings-203121/assets/dynamic-world/v3-5_stack_tests/wri_test_goldsboro')

#Get classes from first image
dw_classes = dynamic_world_classifications_monthly.first().bandNames()
dw_classes_str = dw_classes.getInfo()

#Get dictionary of classes and values
#Define array of land cover classification values
dw_class_values = np.arange(1,10).tolist()
dw_class_values = ee.List(dw_class_values)
#Create dictionary representing land cover classes and land cover class values
dw_classes_dict = ee.Dictionary.fromLists(dw_classes, dw_class_values)


<font size="4">Define color palettes to map land cover</font>

In [ ]:
change_detection_palette = ['#ffffff', # no_data=0
                              '#419bdf', # water=1
                              '#397d49', # trees=2
                              '#88b053', # grass=3
                              '#7a87c6', # flooded_vegetation=4
                              '#e49535', # crops=5
                              '#dfc25a', # scrub_shrub=6
                              '#c4291b', # builtup=7
                              '#a59b8f', # bare_ground=8
                              '#a8ebff', # snow_ice=9
                              '#616161', # clouds=10
]
statesViz = {'min': 0, 'max': 10, 'palette': change_detection_palette};


<font size="4">Gather projection and geometry information from the land cover classifications</font>

In [ ]:
projection = dynamic_world_classifications_monthly.first().projection().getInfo()
crs = projection.get('crs')
crsTransform = projection.get('transform')
scale = dynamic_world_classifications_monthly.first().projection().nominalScale().getInfo()
print('CRS and Transform: ',crs, crsTransform)

geometry = dynamic_world_classifications_monthly.first().geometry().bounds()


<font size="4">Define years to assess land cover and convert monthly classifications to annual classifications</font>

In [ ]:
#Define years to get annual classifications for
years = np.arange(2016,2020)

#Squash scenes from monthly to annual to get the annual classification
dynamic_world_classifications = npv.squashScenesToAnnualClassification(dynamic_world_classifications_monthly,years,method='median')


<font size="4">Define consistent change image to sample training points. Here we will define consistent change as:
* $$\ state(i) != state(i+1) $$
* $$\ state(i+1) = state(i+2) $$
</font>

In [ ]:
#Convert classifications to an image with band names as original image names
dynamic_world_classification_image = dynamic_world_classifications.toBands()

#Get a collection of images where each image has 3 bands: classifications for year(i), classifications for year(i+1), and classifications for year i+2
lc_consistent_change_one_year_col = npv.getYearStackIC(dynamic_world_classification_image,dynamic_world_classification_image.bandNames().getInfo(), band_indices=[0,1,2])

#Get a collection of images where each image represents whether change was consistent from year(i) to year(i+1)
lc_consistent_change_one_year_col = lc_consistent_change_one_year_col.map(npv.LC_ConsistentChangeOneYear)

#Convert this collection to one image
lc_consistent_change_one_year_image = lc_consistent_change_one_year_col.toBands()

#Rename image bands to the central year
lc_consistent_change_one_year_image = lc_consistent_change_one_year_image.rename(lc_consistent_change_one_year_col.aggregate_array('OriginalBand'))



<font size="4">Now *lc_consistent_change_one_year_image* is a binary multi-band image where each band represents
whether change from $year_{i}$ to $year_{i+1}$ is consistent

However it also includes 0's for where there was no change from $year_{i}$ to $year_{i+1}$, therefore we need to
mask the *lc_consistent_change_one_year_image* to only pixels with change form $year_{i}$ to $year_{i+1}$
</font>

<font size="4">Mask consistent change image to pixels with change in that year</font>

In [ ]:
#Get a collection of images where each image has 2 bands: classifications for year(i) and classifications for year(i+1)
lc_one_change_col = npv.getYearStackIC(dynamic_world_classification_image,dynamic_world_classification_image.bandNames().getInfo(), band_indices=[0,1])

#Get a collection of images where each image represents whether there was change from year(i) to year(i+1)
lc_one_change_col = lc_one_change_col.map(npv.LC_OneChange)

#Convert this collection to one image
lc_one_change_image = lc_one_change_col.toBands()

#Rename image bands to the central year
lc_one_change_image = lc_one_change_image.rename(lc_one_change_col.aggregate_array('OriginalBand'))

#Mask each band of the consistent change image to 
lc_consistent_change_one_year_image = lc_consistent_change_one_year_image.mask(lc_one_change_image.select(lc_consistent_change_one_year_image.bandNames()))


<font size="4">Map three years worth of land cover and the consistent change image to check *lc_consistent_change_one_year_image*
</font>

In [ ]:
oneChangeDetectionViz = {'min': 0, 'max': 1, 'palette': ['696a76','ff2b2b']}; #gray = 0, red = 1
consistentChangeDetectionViz = {'min': 0, 'max': 1, 'palette': ['0741df','df07b5']}; #blue = 0, pink = 1


center = [35.410769, -78.100163]
zoom = 12
Map1 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
Map1.addLayer(dynamic_world_classification_image.select('2017_class'),statesViz,name='2017 LC')
Map1.addLayer(dynamic_world_classification_image.select('2018_class'),statesViz,name='2018 LC')
Map1.addLayer(dynamic_world_classification_image.select('2019_class'),statesViz,name='2019 LC')
Map1.addLayer(lc_one_change_image.select('2017_class'),oneChangeDetectionViz,name='One Change from 2017-2018')
Map1.addLayer(lc_consistent_change_one_year_image.select('2017_class'),consistentChangeDetectionViz,
              name='Consistent Change from 2017-2019')
display(Map1)


* The first layer will show classifications for 2017
* The second layer will show classifications for 2018
* The third layer will show classifications for 2019
* The fourth layer will show whether change occured from 2017 to 2018, with grey for no and red for yes
* The fifth layer will show whether change from 2017 to 2018 stayed consistent in 2019, with blue for no and pink for yes
* The final layer should be masked to show only consistent change in the red areas of the fourth layer


## Step 2: Gather neighborhood predictor variables 
<font size="4"> Here we will be collecting predictor variables on seasonal probabilities and the difference in seasonal probabilities from $year_{i}$ to $year_{i+1}$. 
    
For the remainder of the notebook, we'll sample training points and predictor variables for one year only.</font>

In [ ]:
#Calculate seasonal probabilities and the differece in seasonal probabilities from the current year to the following year
#using the getSeasonalDifference function.

#Select year to calculate probabilities and probability differences
year = 2017

#Get band names
dwc_bandNames = dynamic_world_classifications_monthly.first().bandNames().getInfo()

#Caclulate seasonal probabilities and probability differences
#Inputs:
#    probability_collection is the monthly classifications
#    year = 2017
#    band_names = dwc_bandNames, gatehred from dynamic_world_classifications_monthly
#    reduce_method = 'median', where seasonal probabilities are calculated as the median of the monthly probabilities
#    seson_list is used to define the seasons and give them names, including the year, month, and day of the start of the season
#          and the year, month, and day of the end of the season
#    include_difference is set to True to include the difference in seasonal probabilities from year(i) to year(i+year_difference)
#    year_difference is set to 1 to calculate the seasonal difference from year(i) to year(i+1)
#    image_name is used to name the output image using year

#Outputs:
#    an image of seasonal probabilities for each land cover class and the difference in those probabilities
#    from year(i) to year(i+1)

season_changes = npv.getSeasonalDifference(probability_collection = dynamic_world_classifications_monthly, 
                                           year = year, 
                                           band_names = dwc_bandNames, 
                                           reduce_method='median', 
                                           season_list = [['winter',-1,12,1,0,2,'end'],
                                                          ['spring',0,3,1,0,5,'end'],
                                                          ['summer',0,6,1,0,8,'end'],
                                                          ['fall',0,9,1,0,11,'end']],
                                           include_difference=True,
                                           year_difference=1,
                                           image_name = 'season_probs_{}')

#Print the name to see if there are any errors
seasonal_bands = season_changes.bandNames().getInfo()
seasonal_image_name = season_changes.get('system:index').getInfo()
print('Image name: ', seasonal_image_name)
print('Bands: ', seasonal_bands)
print('Number of bands: ', len(seasonal_bands))



<font size="4"> 
Map predictor variables to view any areas of missing data. Looking at the 'Summer Trees' (the probability of trees in the summer) and "Summer Diff Trees' (the difference in probability from 2017-2018 in trees) layers we can see a lot of missing data. These areas will not be sampled from or predicted over using the EE Classifiers.
    </font>

In [ ]:
greenViz = {'min': 0, 'max': 1, 'palette': ['ffffff','16ba21']}; #white = 0, green = 1
blueViz = {'min': 0, 'max': 1, 'palette': ['ffffff','3360ff']}; #white = 0, blue = 1
redViz = {'min': 0, 'max': 1, 'palette': ['ffffff','ff3333']}; #white = 0, blue = 1


center = [35.410769, -78.100163]
zoom = 12
Map2 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
Map2.addLayer(dynamic_world_classification_image.select('2017_class'),statesViz,name='2017 LC')
Map2.addLayer(season_changes.select('winter_start_trees'),greenViz,name='Winter Trees')
Map2.addLayer(season_changes.select('spring_start_trees'),blueViz,name='Spring Trees')
Map2.addLayer(season_changes.select('summer_start_trees'),redViz,name='Summer Trees')
Map2.addLayer(season_changes.select('fall_start_trees'),greenViz,name='Fall Trees')
Map2.addLayer(season_changes.select('winter_difference_trees'),greenViz,name='Winter Diff Trees')
Map2.addLayer(season_changes.select('spring_difference_trees'),blueViz,name='Spring Diff Trees')
Map2.addLayer(season_changes.select('summer_difference_trees'),redViz,name='Summer Diff Trees')
Map2.addLayer(season_changes.select('fall_difference_trees'),greenViz,name='Fall Diff Trees')
display(Map2)


<font size="4">Now we have mutliple predictor variables including:
1. Land cover probabilities for spring, summer, fall and winter in 2017
2. The difference in land cover probabilities from 2017 to 2018 for spring, summer, fall, and winter

More predictor variables could be included by calculating the difference in land cover probabilities from 2016 to 2017 or the difference in land cover probabilities form 2016 to 2018.</font>

In [ ]:
#Define predictor variable image and band names to use in model
#Here we're just using the seasonal probabilities and differences from the cell above
predictor_variable_image = season_changes
predictor_variable_names = seasonal_bands


## Step 3: Gather Training Points

<font size="4">Now that we have a binary image of where consistent change occurs (masked to areas of change) and our predictor variables, we'll build our training, validation, and test set. We'll only sample training points from one year to simplify the process, however you can sample locations from multiple years.

First we'll sample the training point locations by taking stratified samples of the consistent change image, then we'll sample the predictor variables at those locations.</font>

In [ ]:
#Define year for which to sample consistent change, here we'll select 2017
year_to_select = '2017_class'

#We'll remove training points that are within a certain distance to the validation and test points
#However since we're testing over a small area, we'll keep the distance small
distanceToFilter = 100


#Select consistent change for 2017
lc_consistent_change_one_year_select = lc_consistent_change_one_year_image.select(year_to_select).rename('Consistent Change')

#Sample point locations that we will split into training, validation, and test sets
point_locations = npv.getStratifiedSampleBandPoints(lc_consistent_change_one_year_select, region=geometry, 
                                                       numPoints=2000, bandName='Consistent Change',seed=num_seed,
                                                       geometries=True,scale=scale, projection=crs)

#Assign random value between 0 and 1 to split into training, validation, and test sets
point_locations = point_locations.randomColumn(columnName='TrainingSplit', seed=num_seed)
#First split training set from the rest, taking 70% of the points for training
#Roughly 70% training, 30% for validation + testing
training_split = 0.7
training_locations = point_locations.filter(ee.Filter.lt('TrainingSplit', training_split))
validation_and_test = point_locations.filter(ee.Filter.gte('TrainingSplit', training_split))

#Define distance filter to remove training points within a certain distance of test points and validation points
distFilter = ee.Filter.withinDistance(distance=distanceToFilter, leftField='.geo', rightField= '.geo', maxError= 1)
join = ee.Join.inverted()
training_locations = join.apply(training_locations, validation_and_test, distFilter);

#Assign another random value between 0 and 1 to validation_and_test to split to validation and test sets
validation_and_test = validation_and_test.randomColumn(columnName='ValidationSplit', seed=num_seed)
#Of the 30% saved for validation + testing, half goes to validation and half goes to test
#Meaning original sample will be 70% training, 15% validation, 15% testing
validation_split = 0.5 
validation_locations = validation_and_test.filter(ee.Filter.lt('ValidationSplit', validation_split))
test_locations = validation_and_test.filter(ee.Filter.gte('ValidationSplit', validation_split))

#Apply distance filter to remove validation points within a certain distance of test points
validation_locations = join.apply(validation_locations, test_locations, distFilter);

#Export these locations to an Earth Engine asset (we're using the same training point locations as another demo notebook: https://github.com/wri/rw-dynamicworld-cd/blob/master/LandCoverChangeDetection_Example_NeighborhoodPrediction.ipynb)
location_description = '{}_locations'
location_assetID = 'projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_locations'

export_results_task = ee.batch.Export.table.toAsset(
    collection=training_locations, 
    description = location_description.format('training'), 
    assetId = location_assetID.format('training'))
export_results_task.start()
    
export_results_task = ee.batch.Export.table.toAsset(
    collection=validation_locations, 
    description = location_description.format('validation'), 
    assetId = location_assetID.format('validation'))
export_results_task.start()

export_results_task = ee.batch.Export.table.toAsset(
    collection=test_locations, 
    description = location_description.format('test'), 
    assetId = location_assetID.format('test'))
export_results_task.start()
 
#Wait for last export to finish
while export_results_task.active():
    print('Polling for task (id: {}).'.format(export_results_task.id))
    time.sleep(30)
print('Done with export.')


<font size="4">Next we'll sample the predictor variables for the training, validation, and test sets.</font>

In [ ]:
#Define properties to export to an Earth Engine asset
points_description = 'Probability_2017-2018_{}_points'
points_assetID = 'projects/wri-datalab/DynamicWorld_CD/ModelResults/Probability_2017-2018_{}_points'

#Load the point locations assets
training_locations_asset = ee.FeatureCollection('projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_locations'.format('training'))
validation_locations_asset = ee.FeatureCollection('projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_locations'.format('validation'))
test_locations_asset = ee.FeatureCollection('projects/wri-datalab/DynamicWorld_CD/ModelResults/Neighborhood_{}_locations'.format('test'))

#Sample the neighborhood image at point locations 
training_points = predictor_variable_image.sampleRegions(training_locations_asset, scale=scale, projection=crs, geometries=True,tileScale=1)
validation_points = predictor_variable_image.sampleRegions(validation_locations_asset, scale=scale, projection=crs, geometries=True,tileScale=1)
test_points = predictor_variable_image.sampleRegions(test_locations_asset, scale=scale, projection=crs, geometries=True,tileScale=1)

export_results_task = ee.batch.Export.table.toAsset(
    collection=training_points, 
    description = points_description.format('training'), 
    assetId = points_assetID.format('training'))
export_results_task.start()

export_results_task = ee.batch.Export.table.toAsset(
    collection=validation_points, 
    description = points_description.format('validation'), 
    assetId = points_assetID.format('validation'))
export_results_task.start()

export_results_task = ee.batch.Export.table.toAsset(
    collection=test_points, 
    description = points_description.format('test'), 
    assetId = points_assetID.format('test'))
export_results_task.start()

#Wait for export to finish
while export_results_task.active():
    print('Polling for task (id: {}).'.format(export_results_task.id))
    time.sleep(30)
print('Done with export.')
    
    
    
    

## Step 5: define models to test

In [ ]:
#Define dictionaries of parameters and models to test
#You can find the inputs for the parameters under the ee.Classifiers section of GEE

rf_parameters = {'seed':[num_seed], 
          'numberOfTrees': [50,100], 
          'variablesPerSplit': [4,8,10,None], 
          'minLeafPopulation': [None,10,50], 
          'bagFraction': [None,0.5,.3], 
          'maxNodes': [None, 20, 50]
         }
#buildGridSearchList converts the parameter dictionary into a list of classifiers that can be used in cross-validation
rf_classifier_list = gclass.buildGridSearchList(rf_parameters,'smileRandomForest')

svm_parameters = {'decisionProcedure':[None]}
svm_classifier_list = gclass.buildGridSearchList(svm_parameters,'libsvm')

maxent_parameters = {'minIterations':[10,100],
                    'maxIterations':[50,200]}
maxent_classifier_list = gclass.buildGridSearchList(maxent_parameters,'gmoMaxEnt')

classifier_list = rf_classifier_list+svm_classifier_list+maxent_classifier_list



## Step 6: Use cross validation to choose optimal parameters for each model, kernel option


In [ ]:
#Name y_column
y_column = 'Consistent Change'

#Define assetId and description format to export to GEE
cv_results_assetId = 'projects/wri-datalab/DynamicWorld_CD/ModelResults/cv_export_probability_2017-2018'
cv_results_description = 'cv_export_probability_2017-2018'

#Load training points
training_points = ee.FeatureCollection(points_assetID.format('training'))

#Perform cross validation, returns a feature collection
cv_results = gclass.kFoldCrossValidation(inputtedFeatureCollection = training_points, 
                                     propertyToPredictAsString = y_column, 
                                     predictors = predictor_variable_names, 
                                     listOfClassifiers = classifier_list,
                                     k=3,seed=num_seed)
#Export results to GEE
export_results_task = ee.batch.Export.table.toAsset(
        collection=cv_results, 
        description = cv_results_description, 
        assetId = cv_results_assetId)
export_results_task.start()

#Wait for export to finish
while export_results_task.active():
    print('Polling for task (id: {}).'.format(export_results_task.id))
    time.sleep(30)
print('Done with export.')
    


<font size="4">Load cross validation results and find the best model based on accuracy on the validation set</font>

In [ ]:
#Create empty dataframe to save results of cross validation
accuracy_and_keys = pd.DataFrame()

#Load cross-validation results
results = ee.FeatureCollection(cv_results_assetId)
#Get the best result by the cross validation score
best_result = results.sort('Validation Score', False).first()

#Load params as a dictionary
params = best_result.get('Params').getInfo()
params = ast.literal_eval(params)

#Get the calssifier name
classifierName = best_result.get('Classifier Type').getInfo()

#Load classifier with best params
best_model = gclass.defineClassifier(params,classifierName)

#Load training and validation points
training_points = ee.FeatureCollection(points_assetID.format('training'))
validation_points = ee.FeatureCollection(points_assetID.format('validation'))

#Train a classifier with the best params on the training data
best_model = best_model.train(training_points, classProperty=y_column, 
                              inputProperties=predictor_variable_names, subsamplingSeed=num_seed)

#Predict over validation data
validation_points_predicted = validation_points.classify(best_model)

#Get confusion matrix and accuracy score
confusion_matrix = validation_points_predicted.errorMatrix(y_column, 'classification');
accuracy = confusion_matrix.accuracy().getInfo()


#Get confusion matrix and accuracy score
confusion_matrix = validation_points_predicted.errorMatrix(y_column, 'classification');
accuracy = confusion_matrix.accuracy().getInfo()
print('--------------------------------------------------------------')
print('Final Model Validation Set Confusion Matrix')
print(gclass.pretty_print_confusion_matrix(confusion_matrix.getInfo()))
print('Final Model Validation Set Accuracy',accuracy)
print('\n')


#Predict over test data
test_points = ee.FeatureCollection(points_assetID.format('test'))

test_points_predicted = test_points.classify(best_model)

#Get confusion matrix and accuracy score
confusion_matrix = test_points_predicted.errorMatrix(y_column, 'classification');
accuracy = confusion_matrix.accuracy().getInfo()
print('--------------------------------------------------------------')
print('Final Model Test Set Confusion Matrix')
print(gclass.pretty_print_confusion_matrix(confusion_matrix.getInfo()))
print('Final Model Test Set Accuracy',accuracy)
print('\n')


## Step 8: Use model to predict consistent change

In [ ]:
#Use best model and predictor variable image to predict consistent change
predicted_consistent_change = predictor_variable_image.updateMask(lc_one_change_image.select(year_to_select)).classify(best_model)

#The actual consistent change was saved to a variable lc_consistent_change_one_year_select earlier

#Map the results
center = [35.410769, -78.100163]
zoom = 12
Map3 = geemap.Map(center=center, zoom=zoom,basemap=basemaps.Esri.WorldImagery,add_google_map = False)
Map3.addLayer(dynamic_world_classification_image.select('2017_class'),statesViz,name='2017 LC')
Map3.addLayer(dynamic_world_classification_image.select('2018_class'),statesViz,name='2018 LC')
Map3.addLayer(dynamic_world_classification_image.select('2019_class'),statesViz,name='2019 LC')
Map3.addLayer(lc_consistent_change_one_year_select,consistentChangeDetectionViz,
              name='Consistent Change from 2017-2019')
Map3.addLayer(predicted_consistent_change,oneChangeDetectionViz,
              name='Predicted Consistent Change from 2017-2018')

display(Map3)


* The first layer will show classifications for 2017
* The second layer will show classifications for 2018
* The third layer will show classifications for 2019
* The fourth layer will show whether change from 2017 to 2018 stayed consistent in 2019, with grey for no and red for yes with blue for no and pink for yes
* The fifth layer will show the model's prediction on whether change from 2017 to 2018 stayed consistent in 2019, blue for no and pink for yes


In [ ]:
#You can view/save the model using the "explain" function of classifiers
classifierString = best_model.explain().get('trees')
print(classifierString.getInfo())